In [2]:
#pip install langchain_community

In [3]:
#pip install langchain_openai

In [5]:
#pip install Chroma

In [ ]:
# pip install chromadb

In [7]:
#pip install pymupdf

In [8]:
import os
import sys

# Import classes from modules
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [9]:
# Generate API KEY from OPENAI website and define as a variable.
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEYS"

In [10]:
# Define the path to your local PDF file
local_path = "ข้อมูลยา 50 ชนิด.pdf"

# Load and split the PDF file
if local_path:
    loader = PyMuPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file")
    sys.exit()

In [11]:
data

[Document(page_content='นี่คือข้\nอมูลยาสามัญประจำบ้\nาน50 รายการพร้\nอมด้\nวยข้\nอมูลชื่อยา, ส่วนผสม, สรรพคุณ, ขนาดและวิธีใช้\n, คำ\nเตือน, และการเก็บรักษา:\n1. พาราเซตามอล(Paracetamol)\n- ส่วนผสม: พาราเซตามอล\n- สรรพคุณ: บรรเทาอาการปวดและลดไข้\n- ขนาดและวิธีใช้\n: ผู้\nใหญ่500-1000 มิลลิกรัมทุก4-6 ชั่วโมง(ไม่เกิน4 กรัมต่อวัน), เด็กตามคำแนะนำของ\nแพทย์\n- คำเตือน: ห้\nามใช้\nเกินขนาดที่กำหนด, ระวังการใช้\nร่วมกับยาที่มีพาราเซตามอล\n- การเก็บรักษา: เก็บในที่แห้\nงและเย็น, ห่างจากแสงแดด\n2. ไอบูโพรเฟน(Ibuprofen)\n- ส่วนผสม: ไอบูโพรเฟน\n- สรรพคุณ: บรรเทาอาการปวด, ลดไข้\n, และลดการอักเสบ\n- ขนาดและวิธีใช้\n: ผู้\nใหญ่200-400 มิลลิกรัมทุก4-6 ชั่วโมง(ไม่เกิน1.2 กรัมต่อวัน), เด็กตามคำแนะนำของ\nแพทย์\n- คำเตือน: ห้\nามใช้\nในผู้\nที่มีประวัติแพ้\nยาไอบูโพรเฟน, ระวังการใช้\nในผู้\nป\n่วยโรคกระเพาะ\n- การเก็บรักษา: เก็บในที่แห้\nงและเย็น, ห่างจากแสงแดด\n3. ยาลดกรดอลูมิเนียม-แมกนีเซียม(Aluminium-Magnesium Antacid)\n- ส่วนผสม: อลูมิเนียมไฮดรอกไซด์, แมกนีเซียมไฮดรอกไซด์\n- สรรพคุณ: บรรเทาอาการแสบร

In [12]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [13]:
texts

[Document(page_content='นี่คือข้\nอมูลยาสามัญประจำบ้\nาน50 รายการพร้\nอมด้\nวยข้\nอมูลชื่อยา, ส่วนผสม, สรรพคุณ, ขนาดและวิธีใช้\n, คำ\nเตือน, และการเก็บรักษา:\n1. พาราเซตามอล(Paracetamol)\n- ส่วนผสม: พาราเซตามอล\n- สรรพคุณ: บรรเทาอาการปวดและลดไข้\n- ขนาดและวิธีใช้\n: ผู้\nใหญ่500-1000 มิลลิกรัมทุก4-6 ชั่วโมง(ไม่เกิน4 กรัมต่อวัน), เด็กตามคำแนะนำของ\nแพทย์\n- คำเตือน: ห้\nามใช้\nเกินขนาดที่กำหนด, ระวังการใช้\nร่วมกับยาที่มีพาราเซตามอล\n- การเก็บรักษา: เก็บในที่แห้\nงและเย็น, ห่างจากแสงแดด\n2. ไอบูโพรเฟน(Ibuprofen)\n- ส่วนผสม: ไอบูโพรเฟน\n- สรรพคุณ: บรรเทาอาการปวด, ลดไข้\n, และลดการอักเสบ\n- ขนาดและวิธีใช้\n: ผู้\nใหญ่200-400 มิลลิกรัมทุก4-6 ชั่วโมง(ไม่เกิน1.2 กรัมต่อวัน), เด็กตามคำแนะนำของ\nแพทย์\n- คำเตือน: ห้\nามใช้\nในผู้\nที่มีประวัติแพ้\nยาไอบูโพรเฟน, ระวังการใช้\nในผู้\nป\n่วยโรคกระเพาะ\n- การเก็บรักษา: เก็บในที่แห้\nงและเย็น, ห่างจากแสงแดด\n3. ยาลดกรดอลูมิเนียม-แมกนีเซียม(Aluminium-Magnesium Antacid)\n- ส่วนผสม: อลูมิเนียมไฮดรอกไซด์, แมกนีเซียมไฮดรอกไซด์\n- สรรพคุณ: บรรเทาอาการแสบร

In [14]:
# This part is used for embedding the docs and storing them into Vector DB and initializing the retriever.
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [15]:
custom_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI thai language model assistant.
    You are an expert at answering questions about medicine.
    Answer the question based ONLY on the following context.
    


    {context}
    Original question: {question}""",
)

In [16]:
# Create the language model
llm1 = ChatOpenAI(model="gpt-3.5-turbo-16k")

# Create the chain
chain = ConversationalRetrievalChain.from_llm(
    llm=llm1,
    retriever=docsearch.as_retriever(),
    combine_docs_chain_kwargs={"prompt": custom_template}
)

In [17]:
chat_history = []
query = None  # Initialize query to avoid potential reference error

while True:
    if not query:
        query = input("User: ")
    if query in ['quit', 'q', 'exit']:
        break
    result = chain.invoke({"question": query, "chat_history": chat_history})
    print("Chatbot:", result['answer'])

    chat_history.append((query, result['answer']))
    query = None

User:  ยาไรเฟมไพซิน มีส่วนผสมอะไรบ้าง


Chatbot: ยาไรเฟมไพซิน (Rifampicin) มีส่วนผสมเพียงไรเฟมไพซินเท่านั้น


User:  สรรพคุณ


Chatbot: สรรพคุณของยาไรเฟมไพซินคือรักษาโรควัณโรคและโรคติดเชื้ออื่นๆ


User:  มีวิธีใช้ยังไง


Chatbot: วิธีใช้ยาไรเฟมไพซิน (Rifampicin) ในผู้ใหญ่คือ 450-600 มิลลิกรัมต่อวันหนึ่งครั้ง ควรใช้ยาตามคำแนะนำของแพทย์ และควรระวังการใช้ในผู้ป่วยโรคตับ ยาควรเก็บรักษาในที่แห้งและเย็น


User:  ยาริทาเวอรีน มีสรรพคุณยังไง


Chatbot: ริทาเวอรีนมีสรรพคุณในการรักษาการติดเชื้อเอชไอวี โดยมีขนาดและวิธีใช้ผู้ใหญ่คือ 600 มิลลิกรัมวันละครั้ง การใช้ยานี้ควรระวังในผู้ป่วยโรคตับและควรเก็บรักษายาในที่แห้งและเย็น


User:  q
